In [1]:
BBB = [0x00, 0x01, 0x02, 0x03, 0x04, 0x06, 0x3e, 0x3c, 0x08, 0x11, 0x0e, 0x17, 0x2b, 0x33, 0x35, 0x2d, 0x19, 0x1c, 0x09, 0x0c, 0x15, 0x13, 0x3d, 0x3b, 0x31, 0x2c, 0x25, 0x38, 0x3a, 0x26, 0x36, 0x2a, 0x34, 0x1d, 0x37, 0x1e, 0x30, 0x1a, 0x0b, 0x21, 0x2e, 0x1f, 0x29, 0x18, 0x0f, 0x3f, 0x10, 0x20, 0x28, 0x05, 0x39, 0x14, 0x24, 0x0a, 0x0d, 0x23, 0x12, 0x27, 0x07, 0x32, 0x1b, 0x2f, 0x16, 0x22]
IBBB = [0x00, 0x01, 0x02, 0x03, 0x04, 0x31, 0x05, 0x3a, 0x08, 0x12, 0x35, 0x26, 0x13, 0x36, 0x0a, 0x2c, 0x2e, 0x09, 0x38, 0x15, 0x33, 0x14, 0x3e, 0x0b, 0x2b, 0x10, 0x25, 0x3c, 0x11, 0x21, 0x23, 0x29, 0x2f, 0x27, 0x3f, 0x37, 0x34, 0x1a, 0x1d, 0x39, 0x30, 0x2a, 0x1f, 0x0c, 0x19, 0x0f, 0x28, 0x3d, 0x24, 0x18, 0x3b, 0x0d, 0x20, 0x0e, 0x1e, 0x22, 0x1b, 0x32, 0x1c, 0x17, 0x07, 0x16, 0x06, 0x2d]

IPI1 = [3, 0, 4, 19, 20, 23, 2, 1, 5, 18, 21, 22, 6, 9, 10, 13, 14, 17, 7, 8, 11, 12, 15, 16]
IPI2 = [0, 1, 6, 7, 2, 3, 8, 9, 4, 5, 10, 11, 13, 14, 19, 18, 12, 15, 20, 21, 16, 17, 22, 23]
IPI3 = [6, 14, 4, 19, 18, 3, 7, 15, 5, 23, 22, 2, 10, 9, 12, 13, 0, 20, 11, 8, 17, 16, 1, 21]

# Inverse Bit-Permutation with PI2
def IBPL2(x):
    y = [False for _ in range(24)]
    for i in range(0, 24):
        y[i] = x[IPI2[i]]
    for i in range(0, 24):
        x[i] = y[i]

# Inverse Linear Mixing Layer in Data Path: inv_theta_d
def ILML1(x):
    y = [False for _ in range(24)]
    for i in range(0, 24):
        y[i] = x[(i + 8) % 24] ^ x[(i + 20) % 24] ^ x[(i + 22) % 24]
    for i in range(0, 24):
        x[i] = y[i]

# Inverse Bit-Permutation with IPI1
def IBPL1(x):
    y = [False for _ in range(24)]
    for i in range(0, 24):
        y[i] = x[IPI1[i]]
    for i in range(0, 24):
        x[i] = y[i]

def ISBL(x):
    IBBB = [0x00, 0x01, 0x02, 0x03, 0x04, 0x31, 0x05, 0x3a, 0x08, 0x12, 0x35, 0x26, 0x13, 0x36, 0x0a, 0x2c, 0x2e,
            0x09, 0x38, 0x15, 0x33, 0x14, 0x3e, 0x0b, 0x2b, 0x10, 0x25, 0x3c, 0x11, 0x21, 0x23, 0x29, 0x2f, 0x27,
            0x3f, 0x37, 0x34, 0x1a, 0x1d, 0x39, 0x30, 0x2a, 0x1f, 0x0c, 0x19, 0x0f, 0x28, 0x3d, 0x24, 0x18, 0x3b,
            0x0d, 0x20, 0x0e, 0x1e, 0x22, 0x1b, 0x32, 0x1c, 0x17, 0x07, 0x16, 0x06, 0x2d]

    for j in range(0, 4):
        i = 6 * j + 5
        a = 1 if x[i] == 1 else 0
        for k in range(5):
            i -= 1
            a <<= 1
            a ^= 1 if x[i] == 1 else 0
        a = IBBB[a]
        i = 6 * j + 5
        for k in range(6):
            x[i - k] = (a >> (5 - k)) & 1

def IRFC(x):
    IBPL2(x)
    ILML1(x)
    IBPL1(x)
    ISBL(x)

def IBPL3(x):
    y = [0 for _ in range(24)]
    for i in range(0, 24):
        y[i] = x[IPI3[i]]
    for i in range(0, 24):
        x[i] = y[i]


Define Round Key 4 And Initilize Lists

In [2]:
selected_indices = [7, 15, 5, 23, 22, 2]
real_k4 = '000000000011000000011100'
k4_list= []
k4_list_valid = []

for j in range(2**24):
    k4_list.append(format(j, '024b'))

Injecting First Fault

In [3]:
#fault1
correct_round4 = '001110001011010110000010'
faulty_round4  = '111001110000110000100011'

for k4 in k4_list:
    result2 = int(k4, 2) ^ int(faulty_round4, 2)
    result2_h = format(result2, '024b')
    input_list_faulty = [int(bit) for bit in result2_h]
    IRFC(input_list_faulty)
    output_binary_faulty = ''.join(map(str, input_list_faulty))

    # Permutation round 10
    permutation_round10 = [int(bit) for bit in output_binary_faulty]
    IBPL3(permutation_round10)
    output_permutation_round10 = ''.join(map(str, permutation_round10))

    # Extracting sbox2 6 bits
    sbox2_bits_faulty = output_permutation_round10[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_faulty = output_permutation_round10[12:18]


    result2_c = int(k4, 2) ^ int(correct_round4, 2)
    result2_h_c = format(result2_c, '024b')
    input_list_correct = [int(bit) for bit in result2_h_c]
    IRFC(input_list_correct)
    output_binary_correct = ''.join(map(str, input_list_correct))

    # Permutation round 10
    permutation_round10_correct = [int(bit) for bit in output_binary_correct]
    IBPL3(permutation_round10_correct)
    output_permutation_round10_correct = ''.join(map(str, permutation_round10_correct))

    # Extracting sbox2 6 bits
    sbox2_bits_correct = output_permutation_round10_correct[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_correct = output_permutation_round10_correct[12:18]

    # XOR sbox 2
    result_xor = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox2_bits_faulty, sbox2_bits_correct)])
    reversed_result_xor = result_xor[::-1]
    hex_result = hex(int(reversed_result_xor, 2))

    # XOR sbox 3
    result_xor_3 = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox3_bits_faulty, sbox3_bits_correct)])
    reversed_result_xor_3 = result_xor_3[::-1]
    hex_result_3 = hex(int(reversed_result_xor_3, 2))

    if hex_result in ['0x0','0x2', '0x3', '0x6', '0x7', '0xc','0xd','0x10','0x11','0x14','0x15','0x1e','0x1f','0x28','0x29','0x3a','0x3b'] and \
       hex_result_3 in ['0x0','0x1','0x2','0x6','0x8','0xb','0xc','0x10','0x11','0x14','0x1a','0x1b','0x1e','0x22','0x25','0x26','0x28','0x2c','0x2f','0x30','0x34','0x35','0x3a','0x3e','0x3f']:
       k4_list_valid.append(k4)

print("len k4 list valid", len(k4_list_valid))

if real_k4 in k4_list_valid:
    print('round key k4 found!')


len k4 list valid 2847232
round key k4 found!


Injecting Second Fault

In [4]:
#fault 2
correct_round4 = '111100101101010111010010'
faulty_round4  = '001101011111100001110011'
k4_list_valid2=[]
for k4 in k4_list_valid:
    result2 = int(k4, 2) ^ int(faulty_round4, 2)
    result2_h = format(result2, '024b')
    input_list_faulty = [int(bit) for bit in result2_h]
    IRFC(input_list_faulty)
    output_binary_faulty = ''.join(map(str, input_list_faulty))

    # Permutation round 10
    permutation_round10 = [int(bit) for bit in output_binary_faulty]
    IBPL3(permutation_round10)
    output_permutation_round10 = ''.join(map(str, permutation_round10))

    # Extracting sbox2 6 bits
    sbox2_bits_faulty = output_permutation_round10[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_faulty = output_permutation_round10[12:18]

    result2_c = int(k4, 2) ^ int(correct_round4, 2)
    result2_h_c = format(result2_c, '024b')
    input_list_correct = [int(bit) for bit in result2_h_c]
    IRFC(input_list_correct)
    output_binary_correct = ''.join(map(str, input_list_correct))

    # Permutation round 10
    permutation_round10_correct = [int(bit) for bit in output_binary_correct]
    IBPL3(permutation_round10_correct)
    output_permutation_round10_correct = ''.join(map(str, permutation_round10_correct))

    # Extracting sbox2 6 bits
    sbox2_bits_correct = output_permutation_round10_correct[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_correct = output_permutation_round10_correct[12:18]

    # XOR sbox 2
    result_xor = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox2_bits_faulty, sbox2_bits_correct)])
    reversed_result_xor = result_xor[::-1]
    hex_result = hex(int(reversed_result_xor, 2))

    # XOR sbox 3
    result_xor_3 = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox3_bits_faulty, sbox3_bits_correct)])
    reversed_result_xor_3 = result_xor_3[::-1]
    hex_result_3 = hex(int(reversed_result_xor_3, 2))

    if hex_result in ['0x0','0x2', '0x3', '0x6', '0x7', '0xc','0xd','0x10','0x11','0x14','0x15','0x1e','0x1f','0x28','0x29','0x3a','0x3b'] and \
       hex_result_3 in ['0x0','0x1','0x2','0x6','0x8','0xb','0xc','0x10','0x11','0x14','0x1a','0x1b','0x1e','0x22','0x25','0x26','0x28','0x2c','0x2f','0x30','0x34','0x35','0x3a','0x3e','0x3f']:
       k4_list_valid2.append(k4)

print("len k4 list valid", len(k4_list_valid2))
if real_k4 in k4_list_valid2:
    print('round key k4 found!')


len k4 list valid 376496
round key k4 found!


Injecting Third Fault

In [5]:
#fault 3
correct_round4 = '111001101101100101000110'
faulty_round4  = '000101110010110011001001'
k4_list_valid3=[]
for k4 in k4_list_valid2:
    result2 = int(k4, 2) ^ int(faulty_round4, 2)
    result2_h = format(result2, '024b')
    input_list_faulty = [int(bit) for bit in result2_h]
    IRFC(input_list_faulty)
    output_binary_faulty = ''.join(map(str, input_list_faulty))

    # Permutation round 10
    permutation_round10 = [int(bit) for bit in output_binary_faulty]
    IBPL3(permutation_round10)
    output_permutation_round10 = ''.join(map(str, permutation_round10))

    # Extracting sbox2 6 bits
    sbox2_bits_faulty = output_permutation_round10[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_faulty = output_permutation_round10[12:18]


    result2_c = int(k4, 2) ^ int(correct_round4, 2)
    result2_h_c = format(result2_c, '024b')
    input_list_correct = [int(bit) for bit in result2_h_c]
    IRFC(input_list_correct)
    output_binary_correct = ''.join(map(str, input_list_correct))

    # Permutation round 10
    permutation_round10_correct = [int(bit) for bit in output_binary_correct]
    IBPL3(permutation_round10_correct)
    output_permutation_round10_correct = ''.join(map(str, permutation_round10_correct))

    # Extracting sbox2 6 bits
    sbox2_bits_correct = output_permutation_round10_correct[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_correct = output_permutation_round10_correct[12:18]

    # XOR sbox 2
    result_xor = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox2_bits_faulty, sbox2_bits_correct)])
    reversed_result_xor = result_xor[::-1]
    hex_result = hex(int(reversed_result_xor, 2))

    # XOR sbox 3
    result_xor_3 = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox3_bits_faulty, sbox3_bits_correct)])
    reversed_result_xor_3 = result_xor_3[::-1]
    hex_result_3 = hex(int(reversed_result_xor_3, 2))

    if hex_result in ['0x0','0x2', '0x3', '0x6', '0x7', '0xc','0xd','0x10','0x11','0x14','0x15','0x1e','0x1f','0x28','0x29','0x3a','0x3b'] and \
       hex_result_3 in ['0x0','0x1','0x2','0x6','0x8','0xb','0xc','0x10','0x11','0x14','0x1a','0x1b','0x1e','0x22','0x25','0x26','0x28','0x2c','0x2f','0x30','0x34','0x35','0x3a','0x3e','0x3f']:
       k4_list_valid3.append(k4)

print("len k4 list valid", len(k4_list_valid3))

if real_k4 in k4_list_valid3:
    print('round key k4 found!')


len k4 list valid 53328
round key k4 found!


Injecting Fourth Fault

In [6]:
#fault 4
correct_round4 = '011111001001110011110110'
faulty_round4  = '100111000100011110011011'
k4_list_valid4=[]
for k4 in k4_list_valid3:
    result2 = int(k4, 2) ^ int(faulty_round4, 2)
    result2_h = format(result2, '024b')
    input_list_faulty = [int(bit) for bit in result2_h]
    IRFC(input_list_faulty)
    output_binary_faulty = ''.join(map(str, input_list_faulty))

    # Permutation round 10
    permutation_round10 = [int(bit) for bit in output_binary_faulty]
    IBPL3(permutation_round10)
    output_permutation_round10 = ''.join(map(str, permutation_round10))

    # Extracting sbox2 6 bits
    sbox2_bits_faulty = output_permutation_round10[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_faulty = output_permutation_round10[12:18]


    result2_c = int(k4, 2) ^ int(correct_round4, 2)
    result2_h_c = format(result2_c, '024b')
    input_list_correct = [int(bit) for bit in result2_h_c]
    IRFC(input_list_correct)
    output_binary_correct = ''.join(map(str, input_list_correct))

    # Permutation round 10
    permutation_round10_correct = [int(bit) for bit in output_binary_correct]
    IBPL3(permutation_round10_correct)
    output_permutation_round10_correct = ''.join(map(str, permutation_round10_correct))

    # Extracting sbox2 6 bits
    sbox2_bits_correct = output_permutation_round10_correct[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_correct = output_permutation_round10_correct[12:18]

    # XOR sbox 2
    result_xor = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox2_bits_faulty, sbox2_bits_correct)])
    reversed_result_xor = result_xor[::-1]
    hex_result = hex(int(reversed_result_xor, 2))

    # XOR sbox 3
    result_xor_3 = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox3_bits_faulty, sbox3_bits_correct)])
    reversed_result_xor_3 = result_xor_3[::-1]
    hex_result_3 = hex(int(reversed_result_xor_3, 2))

    if hex_result in ['0x0','0x2', '0x3', '0x6', '0x7', '0xc','0xd','0x10','0x11','0x14','0x15','0x1e','0x1f','0x28','0x29','0x3a','0x3b'] and \
       hex_result_3 in ['0x0','0x1','0x2','0x6','0x8','0xb','0xc','0x10','0x11','0x14','0x1a','0x1b','0x1e','0x22','0x25','0x26','0x28','0x2c','0x2f','0x30','0x34','0x35','0x3a','0x3e','0x3f']:
       k4_list_valid4.append(k4)

print("len k4 list valid", len(k4_list_valid4))

if real_k4 in k4_list_valid4:
    print('round key k4 found!')




len k4 list valid 5435
round key k4 found!


Injecting Fifth Fault

In [7]:
#fault 5
correct_round4 = '101011100000010101000111'
faulty_round4  = '001001100111000000001011'
k4_list_valid5=[]
for k4 in k4_list_valid4:
    result2 = int(k4, 2) ^ int(faulty_round4, 2)
    result2_h = format(result2, '024b')
    input_list_faulty = [int(bit) for bit in result2_h]
    IRFC(input_list_faulty)
    output_binary_faulty = ''.join(map(str, input_list_faulty))

    # Permutation round 10
    permutation_round10 = [int(bit) for bit in output_binary_faulty]
    IBPL3(permutation_round10)
    output_permutation_round10 = ''.join(map(str, permutation_round10))

    # Extracting sbox2 6 bits
    sbox2_bits_faulty = output_permutation_round10[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_faulty = output_permutation_round10[12:18]


    result2_c = int(k4, 2) ^ int(correct_round4, 2)
    result2_h_c = format(result2_c, '024b')
    input_list_correct = [int(bit) for bit in result2_h_c]
    IRFC(input_list_correct)
    output_binary_correct = ''.join(map(str, input_list_correct))

    # Permutation round 10
    permutation_round10_correct = [int(bit) for bit in output_binary_correct]
    IBPL3(permutation_round10_correct)
    output_permutation_round10_correct = ''.join(map(str, permutation_round10_correct))

    # Extracting sbox2 6 bits
    sbox2_bits_correct = output_permutation_round10_correct[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_correct = output_permutation_round10_correct[12:18]

    # XOR sbox 2
    result_xor = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox2_bits_faulty, sbox2_bits_correct)])
    reversed_result_xor = result_xor[::-1]
    hex_result = hex(int(reversed_result_xor, 2))

    # XOR sbox 3
    result_xor_3 = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox3_bits_faulty, sbox3_bits_correct)])
    reversed_result_xor_3 = result_xor_3[::-1]
    hex_result_3 = hex(int(reversed_result_xor_3, 2))

    if hex_result in ['0x0','0x2', '0x3', '0x6', '0x7', '0xc','0xd','0x10','0x11','0x14','0x15','0x1e','0x1f','0x28','0x29','0x3a','0x3b'] and \
       hex_result_3 in ['0x0','0x1','0x2','0x6','0x8','0xb','0xc','0x10','0x11','0x14','0x1a','0x1b','0x1e','0x22','0x25','0x26','0x28','0x2c','0x2f','0x30','0x34','0x35','0x3a','0x3e','0x3f']:
       k4_list_valid5.append(k4)

print("len k4 list valid", len(k4_list_valid5))

if real_k4 in k4_list_valid5:
    print('round key k4 found!')


len k4 list valid 961
round key k4 found!


Injecting Sixth Fault

In [8]:
#fault 6
correct_round4 = '001010000110000001110011'
faulty_round4  = '000100111000011001001101'
k4_list_valid6=[]
for k4 in k4_list_valid5:
    result2 = int(k4, 2) ^ int(faulty_round4, 2)
    result2_h = format(result2, '024b')
    input_list_faulty = [int(bit) for bit in result2_h]
    IRFC(input_list_faulty)
    output_binary_faulty = ''.join(map(str, input_list_faulty))

    # Permutation round 10
    permutation_round10 = [int(bit) for bit in output_binary_faulty]
    IBPL3(permutation_round10)
    output_permutation_round10 = ''.join(map(str, permutation_round10))

    # Extracting sbox2 6 bits
    sbox2_bits_faulty = output_permutation_round10[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_faulty = output_permutation_round10[12:18]


    result2_c = int(k4, 2) ^ int(correct_round4, 2)
    result2_h_c = format(result2_c, '024b')
    input_list_correct = [int(bit) for bit in result2_h_c]
    IRFC(input_list_correct)
    output_binary_correct = ''.join(map(str, input_list_correct))

    # Permutation round 10
    permutation_round10_correct = [int(bit) for bit in output_binary_correct]
    IBPL3(permutation_round10_correct)
    output_permutation_round10_correct = ''.join(map(str, permutation_round10_correct))

    # Extracting sbox2 6 bits
    sbox2_bits_correct = output_permutation_round10_correct[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_correct = output_permutation_round10_correct[12:18]

    # XOR sbox 2
    result_xor = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox2_bits_faulty, sbox2_bits_correct)])
    reversed_result_xor = result_xor[::-1]
    hex_result = hex(int(reversed_result_xor, 2))

    # XOR sbox 3
    result_xor_3 = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox3_bits_faulty, sbox3_bits_correct)])
    reversed_result_xor_3 = result_xor_3[::-1]
    hex_result_3 = hex(int(reversed_result_xor_3, 2))

    if hex_result in ['0x0','0x2', '0x3', '0x6', '0x7', '0xc','0xd','0x10','0x11','0x14','0x15','0x1e','0x1f','0x28','0x29','0x3a','0x3b'] and \
       hex_result_3 in ['0x0','0x1','0x2','0x6','0x8','0xb','0xc','0x10','0x11','0x14','0x1a','0x1b','0x1e','0x22','0x25','0x26','0x28','0x2c','0x2f','0x30','0x34','0x35','0x3a','0x3e','0x3f']:
       k4_list_valid6.append(k4)

print("len k4 list valid", len(k4_list_valid6))

if real_k4 in k4_list_valid6:
    print('round key k4 found!')

len k4 list valid 220
round key k4 found!


Injecting seventh Fault

In [9]:
#fault 7
correct_round4 = '000111110011011001100101'
faulty_round4  = '110110100101011111001000'
k4_list_valid7=[]
for k4 in k4_list_valid6:
    result2 = int(k4, 2) ^ int(faulty_round4, 2)
    result2_h = format(result2, '024b')
    input_list_faulty = [int(bit) for bit in result2_h]
    IRFC(input_list_faulty)
    output_binary_faulty = ''.join(map(str, input_list_faulty))

    # Permutation round 10
    permutation_round10 = [int(bit) for bit in output_binary_faulty]
    IBPL3(permutation_round10)
    output_permutation_round10 = ''.join(map(str, permutation_round10))

    # Extracting sbox2 6 bits
    sbox2_bits_faulty = output_permutation_round10[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_faulty = output_permutation_round10[12:18]


    result2_c = int(k4, 2) ^ int(correct_round4, 2)
    result2_h_c = format(result2_c, '024b')
    input_list_correct = [int(bit) for bit in result2_h_c]
    IRFC(input_list_correct)
    output_binary_correct = ''.join(map(str, input_list_correct))

    # Permutation round 10
    permutation_round10_correct = [int(bit) for bit in output_binary_correct]
    IBPL3(permutation_round10_correct)
    output_permutation_round10_correct = ''.join(map(str, permutation_round10_correct))

    # Extracting sbox2 6 bits
    sbox2_bits_correct = output_permutation_round10_correct[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_correct = output_permutation_round10_correct[12:18]

    # XOR sbox 2
    result_xor = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox2_bits_faulty, sbox2_bits_correct)])
    reversed_result_xor = result_xor[::-1]
    hex_result = hex(int(reversed_result_xor, 2))

    # XOR sbox 3
    result_xor_3 = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox3_bits_faulty, sbox3_bits_correct)])
    reversed_result_xor_3 = result_xor_3[::-1]
    hex_result_3 = hex(int(reversed_result_xor_3, 2))

    if hex_result in ['0x0','0x2', '0x3', '0x6', '0x7', '0xc','0xd','0x10','0x11','0x14','0x15','0x1e','0x1f','0x28','0x29','0x3a','0x3b'] and \
       hex_result_3 in ['0x0','0x1','0x2','0x6','0x8','0xb','0xc','0x10','0x11','0x14','0x1a','0x1b','0x1e','0x22','0x25','0x26','0x28','0x2c','0x2f','0x30','0x34','0x35','0x3a','0x3e','0x3f']:
       k4_list_valid7.append(k4)

print("len k4 list valid", len(k4_list_valid7))

if real_k4 in k4_list_valid7:
    print('round key k4 found!')

len k4 list valid 75
round key k4 found!


Injecting eighth Fault

In [10]:
#fault 7
correct_round4 = '100010111111100010011101'
faulty_round4  = '010110111101001001100000'
k4_list_valid8=[]
for k4 in k4_list_valid7:
    result2 = int(k4, 2) ^ int(faulty_round4, 2)
    result2_h = format(result2, '024b')
    input_list_faulty = [int(bit) for bit in result2_h]
    IRFC(input_list_faulty)
    output_binary_faulty = ''.join(map(str, input_list_faulty))

    # Permutation round 10
    permutation_round10 = [int(bit) for bit in output_binary_faulty]
    IBPL3(permutation_round10)
    output_permutation_round10 = ''.join(map(str, permutation_round10))

    # Extracting sbox2 6 bits
    sbox2_bits_faulty = output_permutation_round10[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_faulty = output_permutation_round10[12:18]


    result2_c = int(k4, 2) ^ int(correct_round4, 2)
    result2_h_c = format(result2_c, '024b')
    input_list_correct = [int(bit) for bit in result2_h_c]
    IRFC(input_list_correct)
    output_binary_correct = ''.join(map(str, input_list_correct))

    # Permutation round 10
    permutation_round10_correct = [int(bit) for bit in output_binary_correct]
    IBPL3(permutation_round10_correct)
    output_permutation_round10_correct = ''.join(map(str, permutation_round10_correct))

    # Extracting sbox2 6 bits
    sbox2_bits_correct = output_permutation_round10_correct[6:12]
    # Extracting sbox3 6 bits
    sbox3_bits_correct = output_permutation_round10_correct[12:18]

    # XOR sbox 2
    result_xor = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox2_bits_faulty, sbox2_bits_correct)])
    reversed_result_xor = result_xor[::-1]
    hex_result = hex(int(reversed_result_xor, 2))

    # XOR sbox 3
    result_xor_3 = "".join([str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(sbox3_bits_faulty, sbox3_bits_correct)])
    reversed_result_xor_3 = result_xor_3[::-1]
    hex_result_3 = hex(int(reversed_result_xor_3, 2))

    if hex_result in ['0x0','0x2', '0x3', '0x6', '0x7', '0xc','0xd','0x10','0x11','0x14','0x15','0x1e','0x1f','0x28','0x29','0x3a','0x3b'] and \
       hex_result_3 in ['0x0','0x1','0x2','0x6','0x8','0xb','0xc','0x10','0x11','0x14','0x1a','0x1b','0x1e','0x22','0x25','0x26','0x28','0x2c','0x2f','0x30','0x34','0x35','0x3a','0x3e','0x3f']:
       k4_list_valid8.append(k4)

print("len k4 list valid", len(k4_list_valid8))

if real_k4 in k4_list_valid8:
    print('round key k4 found!')

len k4 list valid 35
round key k4 found!
